# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1o2lpxoi9heyQV1hIlsHXWSfDkBPtze-V?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#abrir um navegador
navegador = webdriver.Chrome()

navegador.get('https://www.google.com.br/')

#abrir e rodar em segundo plano
# from selenium.webdriver.chrome.options import Options
# chrome_options Options()
# chrome_options.headless = True
# nav = webdriver.Chrome(options=chrome_options)

# Passo 1: pegar cotação do dolar
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

# Passo 2: pegar cotação do euro
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: pegar cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

navegador.quit()


# Passo 5: Recalcular o preço de cada produto
# Passo 6: Salvar os novos preços dos produtos


4.9144


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"}
  (Session info: chrome=91.0.4472.114)


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [4]:
# Passo 4: importar a lista de produtos
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [5]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna moeda = dolar

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base  original * cotação)
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]
# atualizar o preço final (preço base reais * Margem)
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]

display(tabela)

NameError: name 'cotacao_euro' is not defined

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
tabela.to_excel("Produtos-novo.xlsx", index=False)